In [ ]:
# mounting drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/BullingerDigitalLMFootnotes/git_repo


/content/drive/MyDrive/NLP_2_Conspiracies/code/task/LM_classification_huggingface


## Preparations

### Installs with pip

In [1]:
%pip install accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
%pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.7 MB/s eta 0:00:00


### Huggingface login

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Imports

In [4]:
from transformers import StoppingCriteria, StoppingCriteriaList
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

### Texts for testing:

In [5]:
# Small input, one FN is there, a second one is to be generated:
text1 = """<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file10224" type="Brief" source="HBBW-3" n="193">
	<teiHeader xml:lang="de">
		<fileDesc>
			<titleStmt>
				<title subtype="file">Konrad Geßner, Johannes Fries / Basel an Heinrich Bullinger, 25. Februar [1533]</title>
			</titleStmt>
			<publicationStmt>
				<authority>Universität Zürich</authority>
				</publicationStmt>
			<sourceDesc>
				</sourceDesc>
		</fileDesc>
		</teiHeader>
	<text xml:lang="la">
		<body>
			<div xml:id="div1" corresp="regest1">
				<p>
					<s n="1" xml:lang="la" type="auto">Optimo et integerrimo viro M. Henrico Bullingero, mecaenati charissimo.</s>
				</p>
				<p>
					<s n="2" xml:lang="la" type="auto">S.</s>
					<s n="3" xml:lang="la" type="auto">Impediunt nos ab itinere<note xml:id="fn3" type="footnote" n="3"><persName ref="p1283" cert="high">Geßner</persName> und <persName ref="p1214" cert="high">Fries</persName> befanden sich auf dem Weg nach <placeName ref="l59" cert="high">Bourges</placeName>.</note> nives, pluvia et ventorum vis.</s>
					<s n="4" xml:lang="la" type="auto"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName><note xml:id="fn4" type="footnote" n="4">"""

# Same Letter, but now the first FN is to be generated. Meaning the model has no direct example of a FN in the edition
text2 = """<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file10224" type="Brief" source="HBBW-3" n="193">
	<teiHeader xml:lang="de">
		<fileDesc>
			<titleStmt>
				<title subtype="file">Konrad Geßner, Johannes Fries / Basel an Heinrich Bullinger, 25. Februar [1533]</title>
			</titleStmt>
			<publicationStmt>
				<authority>Universität Zürich</authority>
				</publicationStmt>
			<sourceDesc>
				</sourceDesc>
		</fileDesc>
		</teiHeader>
	<text xml:lang="la">
		<body>
			<div xml:id="div1" corresp="regest1">
				<p>
					<s n="1" xml:lang="la" type="auto">Optimo et integerrimo viro M. Henrico Bullingero, mecaenati charissimo.</s>
				</p>
				<p>
					<s n="2" xml:lang="la" type="auto">S.</s>
					<s n="3" xml:lang="la" type="auto">Impediunt nos ab itinere<note xml:id="fn3" type="footnote" n="3">"""

# A long letter, the model has to generate the very last FN
text3 =  """<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file12260" type="Brief" source="HBBW-15" n="2213">
	<teiHeader xml:lang="de">
		<fileDesc>
			<titleStmt>
				<title subtype="file">Matthias Erb / Reichenweier an Heinrich Bullinger, 11. August [1545]</title>
			</titleStmt>
			<publicationStmt>
				<authority>Universität Zürich</authority>
				</publicationStmt>
			<sourceDesc>
				<bibl type="scan">Zürich StA</bibl>
				</sourceDesc>
		</fileDesc>
		</teiHeader>
	<text xml:lang="la">
		<body>
			<div xml:id="div1" corresp="regest1">
				<p>
					<s n="1" xml:lang="la" type="auto">Gratia domini tecum.</s>
					<s n="2" xml:lang="la" type="auto">Quum decima augusti<note xml:id="fn2" type="footnote" n="2">An einem Montag, dem Vortag (s. Z. 17f) des Datums dieses Briefes.</note> e suggestu descendissem me actutum a templo ad aulam conferens (vocatus scilicet a cancellario<note xml:id="fn3" type="footnote" n="3"><persName ref="p8103" cert="high">Sigismund Stier</persName>.</note> ob quędam negotia ecclesię componenda), venit hic tuus legatus<note xml:id="fn4" type="footnote" n="4"><persName ref="p18219" cert="high">Markus Wüst</persName>; s. oben <ref target="file12256">Nr. 2209</ref>, Anm. b.</note> hora ferme nona et, dum in angiportu sisto gradum, servum<note xml:id="fn5" type="footnote" n="5">Unbekannt.</note> ad limina de <persName ref="p1010" cert="high">Erbio</persName> rogat, me vero oculis ostiario significante ne proderet, statim digito demonstrat dicens: „Hic est.“</s>
					<s n="3" xml:lang="la" type="auto">Is ergo ad me veniens percunctatur, an mihi sit nomen <persName ref="p1010" cert="high">Erbio</persName>.</s>
					<s n="4" xml:lang="la" type="auto">Aio.</s>
					<s n="5" xml:lang="la" type="auto">Statimque litteras<note xml:id="fn6" type="footnote" n="6">Oben <ref target="file12256">Nr. 2209</ref>.</note> depromens dicebat a te missas.</s>
					<s n="6" xml:lang="la" type="auto">Acceptas legi simul obstupescens tuam, qui alias occupatissimus es, sedulitatem in describendis ex autographo copiis<note xml:id="fn7" type="footnote" n="7">Vgl. oben <ref target="file12256">Nr. 2209</ref>, 15-19.</note>.</s>
					<s n="7" xml:lang="la" type="auto">Stans ergo attonitus et propemodum exanimis iussi adolescentem expectare, dum redeam, atque subinde inter tractatus et varias dissidentium quęstiones et tacite et indesinenter volvo ac revolvo intime, quę scripseras.</s>
					<s n="8" xml:lang="la" type="auto">Absolutus a strepitu aulico eo cum adolescente pransum interque epulandum de singulis rebus percunctor, statim scilicet a mensa tibi responsurus.</s>
					<s n="9" xml:lang="la" type="auto">Dum id meditor, alius adest nuntius<note xml:id="fn8" type="footnote" n="8">Unbekannt.</note> litteras ferens a principe<note xml:id="fn9" type="footnote" n="9"><persName ref="p8098" cert="high">Graf Georg von Württemberg-Mömpelgard</persName>, der sich laut Z. 16 damals in <placeName ref="l569" cert="high">Worms</placeName> aufhielt. Seine Teilnahme am <placeName ref="l569" cert="high">Wormser</placeName> Reichstag ist in RTA JR XVIII-2 nicht belegt.</note>, in quibus multa mihi committuntur negotia principemque a comitiis<note xml:id="fn10" type="footnote" n="10">Der Reichstag in <placeName ref="l569" cert="high">Worms</placeName>.</note> propediem rediturum pollicentes.</s>
					<s n="10" xml:lang="la" type="auto">Hinc eo die, tametsi adolescens crebro urgeret abituriens, nil scripsi omnia in posterum, hoc est in hodiernum, reiiciens diem.</s>
					<s n="11" xml:lang="la" type="auto">Haec de pueri tui diligentia.</s>
				</p>
				<p>
					<s n="12" xml:lang="la" type="auto">Quod vero ad negotium adtinet, paucis haec accipe.</s>
					<s n="13" xml:lang="la" type="auto">Fateor me iam annum quinquagesimum egressum<note xml:id="fn11" type="footnote" n="11"><persName ref="p1010" cert="high">Erb</persName> war 1494 geboren.</note>, et qui nunc in ea aetate sum, quę sedatior esse cupit, sepicule curas aulicas, quibus distringor,<pb type="scan" next="257v."/> depręcatum, sed parum promovisse; imo, quo plus me expedire cupio, eo amplius perstringor dispendio et vitae et rerum.</s>
					<s n="14" xml:lang="la" type="auto">Sic deo optimo maximo visum est.</s>
				</p>
				<p>
					<s n="15" xml:lang="la" type="auto">Quod ad vitam, ut omnia manifestius depingam, subit tarda senectus,<note xml:id="fn12" type="footnote" n="12"><hi>Vergil,</hi> Aeneis, 9, 610.</note> quę difficilis incipit esse, sola non suffitiens laboribus, dum indies creberrime exantlatur.</s>
					<s n="16" xml:lang="la" type="auto">Nullum habeo symmistam, ut nosti, opinor.</s>
					<s n="17" xml:lang="la" type="auto">Plures iussu principis vocavi; sed horrent id, quod ego quęror.</s>
					<s n="18" xml:lang="la" type="auto">Metuunt enim sibi ab ingeniis cervinis<note xml:id="fn13" type="footnote" n="13">ingenium cervinum: ein ängstliches Wesen; s. <bibl>Adagia</bibl>, 2, 7, 36 (<bibl>ASD</bibl> II/4 110, <ref target="file11679">Nr. 1636</ref>).</note>, nec ego quenquam nisi volentem ad me trahere volo, etc.<pb type="edition" next="439"/></s>
					<s n="19" xml:lang="la" type="auto">Quod vero ad res meas adtinet, male collocat ille, quem ex nomine nosti, stipendia.</s>
					<s n="20" xml:lang="la" type="auto">Ex patrimonio intra quinquennium<note xml:id="fn16" type="footnote" n="16">D.h. seit 1539, was mit dem Amtsantritt <persName ref="p17986" cert="high">Henmans</persName> übereinstimmt.</note> ducentos fere aureos non in meam, sed magis ecclesię necessitatem exposui, quod dies domini revelabit.</s>
					<s n="21" xml:lang="la" type="auto">Nec quęror, optime frater, de perditis rebus, sed de Harpiis furacibus, qui recte monentes non exaudiunt et ecclesię avaritię notam inurunt; quod non adeo magni referret, nisi metuendum de excidio foret.</s>
					<s n="22" xml:lang="la" type="auto">Quamdiu rapiendum, placet Christus; dum defecerit ecclesię fiscus, non placebit ultra Christus!</s>
					<s n="23" xml:lang="la" type="auto">Hactenus 3 fuimus in isto oppido, ubi quisque designatum habuit<pb type="edition" next="440"/> stipendium.</s>
					<s n="24" xml:lang="la" type="auto">Ego nunc solus omnibus curis pręfectus aegre a satrapis<note xml:id="fn18" type="footnote" n="18">Zu verstehen: von den Beamten, u.a. vom neuen Schaffner.</note> meum recipio stipendium, et si egerem, vix triobulo sublevarent.</s>
					<s n="25" xml:lang="la" type="auto">Sunt pręterea et alię rationes, quibus te nolo nunc gravare scribendo, etc.</s>
					<s n="26" xml:lang="la" type="auto">Atque inter tot turbines nondum video satis iustam occasionem desertionis huius ecclesię.</s>
				</p>
				<p>
					<s n="27" xml:lang="de" type="auto"><pb type="scan" next="257a,r."/>Summa.</s>
					<s n="28" xml:lang="la" type="auto">Ita haereo inter sacrum et saxum<note xml:id="fn19" type="footnote" n="19">Siehe Adagia, 1, 1, 15 (<bibl>ASD</bibl> II/1 128f, Nr. 15).</note>, ut nunc sine discrimine ecclesięque Christi maxima iactura, ut facile intelligis, discedere nequeam.</s>
				</p>
				<p>
					<s n="29" xml:lang="la" type="auto">Atque eę sunt potissimę rationes.</s>
					<s n="30" xml:lang="la" type="auto">Ecclesias nostras iam a sexto anno<note xml:id="fn20" type="footnote" n="20">D.h. seit 1538, als Erbin Reichenweier tätig wurde.</note> utcunque in ordinem, ut potuimus - quando, ut volueramus, non licuit - redegimus.</s>
				</p>
				<p>
					<s n="31" xml:lang="la" type="auto">Annitendum ergo, ut cępta perficiantur.</s>
					<s n="32" xml:lang="la" type="auto">Ad hoc ecclesia illa nobis contigua, Montbelgardiana, cum suo principe<note xml:id="fn21" type="footnote" n="21"><persName ref="p1546" cert="high">Graf Christoph von Württemberg</persName>.</note> adhuc vacillat; de qua totus liber foret perscribendus.</s>
					<s n="33" xml:lang="la" type="auto">Pręterea scholę quattuor in isto parviusculo agro sunt constitutę et quinta in rure instituenda, stipendia quoque studiosis quibusdam ante paucos menses pollicita,<note xml:id="fn23" type="footnote" n="23">Erst 1555 kam es zur Einrichtung eines solchen Stipendiums durch Graf Georg; s. <hi>Adam,</hi> aao, S. 306.</note> sed haec non omnia firmata et rata, proponenda etiamnum ecclesię quędam decreta, quibus ad disciplinam et sanctos mores utcunque redigantur, idque statim redeunte principe.</s>
					<s n="34" xml:lang="la" type="auto">Illo enim absente haec omnia conscripta sunt.</s>
					<s n="35" xml:lang="la" type="auto">Dum igitur nunc mutavero locum omnia sub cultro relinquens<note xml:id="fn24" type="footnote" n="24">omnia sub cultro relinquens: alles der Gefahr aussetzend; vgl. <bibl>Adagia</bibl>, 2, 10, 83 (<bibl>ASD</bibl> II/4 328, <ref target="file12030">Nr. 1983</ref>).</note>, facile intelligis, quid periculi inmineat.</s>
					<s n="36" xml:lang="la" type="auto">In primis severus princeps, a quo omnia pendent, facile vel mutaretur vel irritaretur, ut solet hoc hominum genus; id quod nimirum magis in ecclesię quam in meum incomodum cęderet.</s>
					<s n="37" xml:lang="la" type="auto">Viden inter coelum et terram me suspensum palpitare neque posse hac ratione vel vocatione, quę tamen et sancta et iusta, liberari?</s>
					<s n="38" xml:lang="la" type="auto">Occasio mihi expectanda comodior, quę me liberet sine ecclesię iactura.</s>
					<s n="39" xml:lang="la" type="auto">Nam si iam abirem, ecclesia viduata absque pastore inter cunctatores et cessatores fluctuaret, etc.</s>
				</p>
				<p>
					<s n="40" xml:lang="la" type="auto"><pb type="scan" next="257a,v."/>Ago vero et habeo tibi gratias inmortales, humanissime Bullingere, quod admodum ingenti cura meo nomine laboraveris, atque hoc impensius te amo, quod charitatis offitium non fucate, sed dextre candideque in me collocaris, imo significaris in hac sancta vocatione.</s>
					<s n="41" xml:lang="la" type="auto">Respondebit olim procul dubio deus optimus maximus tuis votis, quo <placeName ref="l1976" cert="high">Augustani</placeName> nanciscantur virum aliquem sanctum, ecclesię dignum ministrum,<note xml:id="fn25" type="footnote" n="25"><persName ref="p8018" cert="high">Johannes Haller</persName> wurde schließlich aus Zurich nach <placeName ref="l1976" cert="high">Augsburg</placeName> gesandt; s. oben <ref target="file12211">Nr. 2164</ref>, Anm. 10.</note> atque ego etiam tuis assiduis<pb type="edition" next="441"/> praecibus vel liberer publica occasione vel sub onere non defitiam.</s>
					<s n="42" xml:lang="la" type="auto">Sed te etiamnum per Iesum Christum, ne deinceps plus mihi vel tribuas, immodicus scilicet virtutum mearum, quibus penitus careo, pręco, vel magis sentias de me, quam res est.</s>
					<s n="43" xml:lang="la" type="auto">Optime enim mihi ipsi conscius sum curtę suppellectilis,<note xml:id="fn26" type="footnote" n="26"><hi>Persius,</hi> Saturae, 4, 52.</note> quam tu, ni fallor, maiori atque ego, nescio quorum iuditio persuasus, aestimas.</s>
				</p>
				<p>
					<s n="44" xml:lang="la" type="auto">Porro, quod nunc temporis importunitas negat, alias benignus deus, ecclesiarum verus et unicus pastor<note xml:id="fn27" type="footnote" n="27">Vgl. <cit type="bible"><ref>Hebr 13, 20</ref></cit>; <cit type="bible"><ref>1Petr 2, 25</ref></cit>; <cit type="bible"><ref>5, 4.</ref></cit></note>, solita clementia sancta resarciet oportunitate.</s>
					<s n="45" xml:lang="la" type="auto">Sancto quoque magistratui <placeName ref="l1976" cert="high">Augustano</placeName>, inprimis vero erudito viro domino <persName ref="p8354" cert="high">Georgio Leto</persName><note xml:id="fn28" type="footnote" n="28"><persName ref="p8354" cert="high">Georg Frölich</persName>.</note>, ingentes gratias meo nomine agas, obsecro, quod me, ineptum homuntionem,<note xml:id="fn29" type="footnote" n="29">Vgl. <cit type="bible"><ref>1Kor 15,9</ref></cit>; <cit type="bible"><ref>Eph 3, 8</ref></cit>.</note> adeo egregia dignati sint vocatione.</s>
					<s n="46" xml:lang="la" type="auto">Ego quoque deinceps eius ecclesię maiorem habebo - si potis fuero - ubique rationem, optoque illis stabilem pacem et concordiam talesque antistites, qui pietate, eruditione, prudentia, vigilantia et fidelitate ita pręmineant, quo Christus in ea republica ita mittat radices, ut nullis satanę et impiorum imposturis deiiciatur.</s>
					<s n="47" xml:lang="la" type="auto">Monebis ergo illos, ne me deinceps expectent; ignoro enim meam liberationem.</s>
					<s n="48" xml:lang="la" type="auto">Servet te Christus.</s>
				</p>
				<p>
					<note type="attachment">[Beilage:]</note>
					<s n="49" xml:lang="la" type="auto"><pb type="scan" next="258r."/>Nova: Absolutionem conmitiorum<note xml:id="fn30" type="footnote" n="30">Der Reichstag zu <placeName ref="l569" cert="high">Worms</placeName>.</note> non ignoras, in quibus nil certi definitum, nisi quod istis hybernis mensibus adversarii nobis pacem condixerunt; futura aestate nobis providendum erit<note xml:id="fn31" type="footnote" n="31">Ein Gerücht.</note>.</s>
					<s n="50" xml:lang="la" type="auto">Acta comitiorum nondum sunt publicata.</s>
					<s n="51" xml:lang="la" type="auto">Aiunt quidam caesari<note xml:id="fn33" type="footnote" n="33"><persName ref="p18538" cert="high">Karl V.</persName></note> omnia esse commissa, ut dicat sententiam etiam in religionis negotio certis tamen conditionibus; quodsi verum fuerit, ovem dicemus lupo<note xml:id="fn34" type="footnote" n="34">Vgl. <cit type="bible"><ref>Joh 10, 12</ref></cit>.</note> commissam.</s>
				</p>
				<p>
					<s n="52" xml:lang="la" type="auto"><persName ref="p18477" cert="high">Ferdinandus</persName> priori septimana<note xml:id="fn35" type="footnote" n="35">Zu verstehen: In der ersten Augustwoche. - <persName ref="p18477" cert="high">Ferdinand I.</persName> war am 31. Juli 1545 aus <placeName ref="l569" cert="high">Worms</placeName> abgereist; s. unten <ref target="file12261">Nr. 2214</ref>, 28 und Anm. 20.</note> a comitiis magno strepitu solvens Bohemiam petit cum <persName ref="p7688" cert="high">Nausea</persName><note xml:id="fn36" type="footnote" n="36"><persName ref="p7688" cert="high">Friedrich Nausea</persName>, Bischof von <placeName ref="l546" cert="high">Wien</placeName>.</note> suo et quibusdam monachis.</s>
					<s n="53" xml:lang="la" type="auto">Sequenti septimana<pb type="edition" next="442"/> caesar abiturus est, sed quo, ignoratur<note xml:id="fn37" type="footnote" n="37"><persName ref="p18538" cert="high">Karl V.</persName> hatte <placeName ref="l569" cert="high">Worms</placeName> am 7. August in Richtung <placeName ref="l267" cert="high">Köln</placeName> verlassen; s. oben <ref target="file12261">Nr. 2214</ref>, 31 und Anm. 23f.</note>.</s>
					<s n="54" xml:lang="la" type="auto">Erant alia dicta comitia ad <placeName ref="l410" cert="high">Reginospurgum</placeName> ad mensem octobrem,<note xml:id="fn38" type="footnote" n="38">Der geplante Reichstag in <placeName ref="l410" cert="high">Regensburg</placeName>, der erst Anfang 1546 begann; s. oben <ref target="file12258">Nr. 2211</ref>, Anm. 24.</note>, sed aiunt quidam irritata<note xml:id="fn39" type="footnote" n="39">Hier im Sinne von "vergeblich"; s. <hi><persName ref="p8320" cert="high">Dasypodius</persName>,</hi> Dic. 199.</note>, etc.</s>
				</p>
				<p>
					<s n="55" xml:lang="la" type="auto">Urgent plures nacionale concilium, idque <placeName ref="322" type="auto_name">Metis</placeName><note xml:id="fn40" type="footnote" n="40"><placeName ref="l322" cert="high">Metz</placeName>.</note> celebrari postulant.</s>
					<s n="56" xml:lang="la" type="auto">Musher<note xml:id="fn41" type="footnote" n="41">Vom französischen "Monsieur".</note> de Gyß<note xml:id="fn42" type="footnote" n="42"><persName ref="p18450" cert="high">Claude I. von Lothringen</persName>, Herzog von Guise.</note>, defuncti ducis Lotharingię frater<note xml:id="fn43" type="footnote" n="43">Der am 12. Juni 1545 verstorbene <persName ref="p7637" cert="high">François I. von Lothringen</persName> war nicht Claudes Bruder, sondern sein Neffe. Bruder von Claude war der Vater des Verstorbenen, nämlich <persName ref="p18431" cert="high">Anton II. von Lothringen</persName> (gest. Juni 1544).</note>, patruum suum conscripto milite et auxilio regis<note xml:id="fn45" type="footnote" n="45"><persName ref="p7637" cert="high">König Franz I.</persName></note> Gallorum conatur e ducatu pellere.</s>
					<s n="57" xml:lang="la" type="auto"><persName ref="p18538" cert="high">Caesar</persName> vero iuniori<note xml:id="fn46" type="footnote" n="46"><persName ref="p18536" cert="high">Karl III. von Lothringen</persName>.</note>, qui patri<note xml:id="fn47" type="footnote" n="47"><persName ref="p7637" cert="high">François I. von Lothringen</persName>.</note> successit, subsidium et <placeName ref="l29" cert="high">Hispanum</placeName> militem pollicitus est.</s>
					<s n="58" xml:lang="la" type="auto">Quodsi res non fuerit composita, putant novum bellum inter caesarem et Gallum oriturum.</s>
					<s n="59" xml:lang="la" type="auto">Feruntur alia quedam nova, sed forte non fide digna: mundus mendatiis scatet<note xml:id="fn48" type="footnote" n="48">Vgl. <bibl>TPMA</bibl> XIII 46, 125f: Die Welt ist falscher Zungen voll.</note>.</s>
					<s n="60" xml:lang="la" type="auto">Haec omnia raptim.</s>
				</p>
				<p>
					<s n="61" xml:lang="la" type="auto">Valebis igitur, vir humanissime, cum ecclesia sancta et familia.</s>
					<s n="62" xml:lang="la" type="auto">Salutabis vicissim tuos, in primis meum <persName ref="p8412" cert="high">Megandrum</persName>, <persName ref="p8437" cert="high">Erasmum</persName>,<note xml:id="fn49" type="footnote" n="49">"""

## Run completion model, LLama 3 8B
Prompting the model with the letter up until the start tag of the FN
and stopping the generation when ```</note>``` is generated.

In [5]:
# Functions to generate

class StoppingCriteriaSub(StoppingCriteria):
    """Subclass of StoppingCriteria that allows stopping based on a specific word
    disregarding the exact tokenization. A list of words can be passed to the stops argument.
    Note that it might not work well for longer words"""
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = stops

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            text = tokenizer.decode(input_ids[0])
            # Todo: find a nice way to watch the generation
            # print(text.split()[-1])
            # need to use the text, as the same text can be tokenized differently depending on context...
            # no guarantee that it ends with the text, as end token may be ">." or something...
            if stop in text[-10:]:
                return True
        return False

stop_words = ["</note>"]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words)])


def generate(prompt:str, model, tokenizer):
    input_ids = tokenizer(prompt, return_tensors="pt")['input_ids'].to(model.device)
    # terminators = tokenizer.encode("</note>", add_special_tokens=False)
    # print(terminators)
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            do_sample=True,
            # eos_token_id=terminators,
            temperature=0.7,
            max_new_tokens=500,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.eos_token_id,
            stopping_criteria=stopping_criteria  # stop when "</note>" is generated
            )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [6]:
# load the model


model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Quantization, as shown here: https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing#scrollTo=VPD7QS_DR-mw
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

#### Test 1
Testing the completion of an FN that is not the first one. So the model has an
example of what we want...

Original FN: <br>
```xml
<persName ref="p1214" cert="high">Fries</persName>
und
<persName ref="p1283" cert="high">Geßner</persName>
waren in einem Brief
<persName ref="p8127" cert="high">Pellikans</persName>
vom 19. Februar 1533 an
<persName ref="p8418" cert="high">Myconius</persName>
empfohlen worden (Zürich StA, E II 358, 98).
</note>
```

In [14]:
for i in range(10):
  print(generate(text1, model, tokenizer))
torch.cuda.empty_cache()

In der Druckversion von 1838 ist Myconium in Myconii. In der Druckversion von 1904 ist Myconium in Myconio.</note></
Die Schreibweise ist ungewöhnlich. Die Schreibung des Namens mit "m" ist in der Literatur nicht bekannt. Myconium war ein Basler Drucker.</note>
<persName ref="p8418" cert="high">Myconius</persName> (um 1495–1544) war Professor der Theologie in <placeName ref="l59" cert="high">Bourges</placeName>. Er hatte eine reformatorische Richtung vertreten und gehörte zu den Unterzeichnern des <term ref="l34">Bourges-Eides</term> von 1534. Auf ihn geht die erste lateinische Übersetzung der <ref target="l27">Heidelberger Katechismus</ref> zurück. <persName ref="p1283" cert="high">Geßner</persName> und <persName ref="p1214" cert="high">Fries</persName> wurden durch ihn mit <ref target="l27">Bullinger</ref> in Verbindung gebracht.</note>
Zitat aus <bibl ref="b192">Bullingers Briefwechsel</bibl>, Bd. 1, S. 69.</note></
<persName ref="p8418" cert="high">Myconium</persName> ist der Basle

#### Test 2
Same letter, but the model has to generate the first FN, i.e. it has no example of what a FN in the edition looks like in the prompt.

Original FN: <br>
```xml
<persName ref="p1283" cert="high">Geßner</persName>
und
<persName ref="p1214" cert="high">Fries</persName>
befanden sich auf dem Weg nach
<placeName ref="l59" cert="high">Bourges</placeName>
.
</note>
```


In [15]:
for i in range(10):
  print(generate(text2, model, tokenizer))
torch.cuda.empty_cache()

Eigene Schrift.</note>
Lassen wir den Weg frei.</note>
Vermutlich wegen der von Bullinger verhängten Schreibverbot für die Anhänger des Zürcher Täufertums (vgl. HBBW-3 190). </note>
P. 4. (Bullinger).</note>

[Bullinger 1926, p. 26]</note>
Glossar: Impedire = in irremediale Schwierigkeiten geraten.</note>
Cf. <ptr target="file10223#div2" type="copy"/> (n. 3).</note>
Bullinger, Epist. 26, 4.</note>
*<hi rend="sup">†</hi></note>
[A. 20.]</note><


Test mit einem langen Brief: <br>
- Es geht ein bisschen länger, und braucht c.a. 12 GB RAM (T4 hat bis zu 15)

Original FN: <br>
```xml
<note xml:id="fn49" type="footnote" n="49">
<persName ref="p8437" cert="high">Erasmus Schmid</persName>
.
</note>
```

In [19]:
for i in range(10):
  print(generate(text3, model, tokenizer))
torch.cuda.empty_cache()

Vgl. <bibl>Adagia</bibl>, 4, 1, 84 (<bibl>ASD</bibl> II/5 157, <ref target="file12185">Nr. 2031</ref>). - <persName ref="p8412" cert="high">Megander</persName> und <persName ref="p8437" cert="high">Erasmus</persName> sind offenbar identisch.</note>
Zu verstehen: <persName ref="p8437" cert="high">Erasmus</persName> von <placeName ref="l1398" cert="high">Burg</placeName>. - <persName ref="p8412" cert="high">Johannes Megander</persName>.</note>
D.h. <persName ref="p8437" cert="high">Johann Eschmann</persName>.</note>
Vgl. oben <ref target="file12256">Nr. 2209</ref>, Anm. 14.</note>
Vgl. <bibl>TPMA</bibl> XII 75, 96, 99f: Megander, Erasmus.</note>
Vgl. <bibl>Adagia</bibl>, 1, 4, 4 (<bibl>ASD</bibl> II/1 158, <ref target="file12195">Nr. 1939</ref>).</note>
Erasmus Megander.</note>
Vgl. <bibl>Adagia</bibl>, 1, 3, 2 (<bibl>ASD</bibl> I/1 141, <ref target="file11509">Nr. 1337</ref>).</note>
Vgl. <bibl>Adagia</bibl>, 1, 8, 76 (<bibl>ASD</bibl> II/1 238f, Nr. 176).</note>
Vgl. oben <ref target="

In [22]:
# del model
torch.cuda.empty_cache()

## Run instruction model LLama3 8B

In [20]:


ICL_SYSTEM_PROMPT = "You are a research assistant for a historian, specialized on the European reformation working on an edition of the correspondence of Heinrich Bullinger. Given a letter in TEI format, your task is to complete the missing footnote"

# without an example it is actually rather difficult to get the model to reply in the correct format... idk. I think there is a way to "force output format?" Constrained Decoding...
SYSTEM_PROMPT = """
You are a research assistant for a historian specialized on the European reformation, working on an edition of the correspondence of Heinrich Bullinger.
Given a letter in TEI format, your task is to complete the missing Footnote. That is complete until you reach the </note> tag.
"""
# footnote for the icl
footnote_text2 = """<note xml:id="fn3" type="footnote" n="3"><persName ref="p1283" cert="high">Geßner</persName> und <persName ref="p1214" cert="high">Fries</persName> befanden sich auf dem Weg nach <placeName ref="l59" cert="high">Bourges</placeName>.</note>"""

def instruction_prompt(text):
  """Generate a prompt carefully explaining the task, no examples needed (hopefully...)"""
  messages = [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "researcher", "content": text},
      {"role": "assistant", "content": '<note xml:id="fn4" type="footnote" n="4">'}
  ]
  return messages

def icl_prompt(text):
  """Generate a prompt that gives the example from text2"""
  messages = [
      {"role": "system", "content": ICL_SYSTEM_PROMPT},
      {"role": "historian", "content": text2},
      {"role": "assistant", "content": footnote_text2},
      {"role": "historian", "content": text}
  ]
  return messages


def generate_chat(messages:list, model, tokenizer):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # generate the attention mask, got a warning that it might be needed, though I am not sure if that is necessary
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [8]:
# load the model


model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Quantization, as shown here: https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing#scrollTo=VPD7QS_DR-mw
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Trying out the icl, where we give the first FN as example. The output format is great, but I the FN quality is not really the best, I fear

Original FN: <br>
```xml
<persName ref="p1214" cert="high">Fries</persName>
und
<persName ref="p1283" cert="high">Geßner</persName>
waren in einem Brief
<persName ref="p8127" cert="high">Pellikans</persName>
vom 19. Februar 1533 an
<persName ref="p8418" cert="high">Myconius</persName>
empfohlen worden (Zürich StA, E II 358, 98).
</note>
```
FN from the ICL:<br>
```xml
<note xml:id="fn49" type="footnote" n="49">
<persName ref="p8437" cert="high">Erasmus Schmid</persName>
.
</note>
```

In [12]:

for i in range(10):
  print(generate_chat(icl_prompt(text1), model, tokenizer))
torch.cuda.empty_cache()

<note xml:id="fn4" type="footnote" n="4">v. <bibl>Basel, 1533</bibl>, <bibl>Basel, 1533</bibl>, <bibl>Basel, 1533</bibl>, <bibl>Basel, 1533</bibl>. <ref target="#bibl1" corresp="bibl1">[1]</ref></note>
<note xml:id="fn4" type="footnote" n="4">vgl. <bibl><author ref="p8418" cert="high">Myconius</author>, <title ref="l28" cert="high">Baselische Chronik</title>, <date ref="l28" cert="high">1533</date>, <page ref="l28" cert="high">A2v</page>.</bibl></note>
<note xml:id="fn4" type="footnote" n="4"><ref target="#l28" corresp="l28">Basel</ref> am <ref target="#p8418" corresp="p8418">Mühlenbrunnen</ref>.</note>
<note xml:id="fn4" type="footnote" n="4"><ref target="#l28"><bibl>Basileae</bibl></ref> <ref target="#p8418"><bibl>Myconium</bibl></ref> <date value="1533-02-25" cert="high">25. Februar [1533]</date></note>
<note xml:id="fn4" type="footnote" n="4">v. <bibl ref="Bullinger, Heinrich, 1504-1575. Briefwechsel, Bd. 3. Basel, 1849, S. 128</bibl>.</note>
<note xml:id="fn4" type="footnote" n="4

In [13]:
torch.cuda.empty_cache()

instruction prompt gives whaky results... tinkered a bit with it, but to no avail...

In [21]:
for i in range(10):
  print(generate_chat(instruction_prompt(text1), model, tokenizer))
torch.cuda.empty_cache()

</note>
				</p>
			</div>
		</body>
	</text>
</TEI>

I will complete the missing footnote until the </note> tag:

<persName ref="p8418" cert="high">Myconium</persName> was a printer and bookseller in Basel, and it is likely that the letter was written from his shop.</note>
</p>
</div>
</body>
</text>
</TEI>
itinerantem nostrum, ut inibi <date value="25-02-1533" cert="high">25. Februar [1533]</date> convenirent.
</note>
</note>
was expected.
</note>
                </p>
            </div>
        </body>
    </text>
</TEI>

I will complete the footnote until the </note> tag.

<persName ref="p8418" cert="high">Myconium</persName> was a printer and bookseller in Basel, and <placeName ref="l28" cert="high">Basileae</placeName> is the Latin name for Basel.
was the 25th of February, and we were not able to set out on our journey, hindered as we were by snow, rain, and the force of the winds. From Basel, at the house of Myconius.</note>
</p>
</div>
</body>
</text>
</TEI>


KeyboardInterrupt: 

In [25]:
torch.cuda.empty_cache()

When the example is from another letter, the model is very confused...

In [24]:
for i in range(10):
  print(generate_chat(icl_prompt(text3), model, tokenizer))
torch.cuda.empty_cache()

<note xml:id="fn3" type="footnote" n="3"><persName ref="p1283" cert="high">Geßner</persName> and <persName ref="p1214" cert="high">Fries</persName> were on their way to <placeName ref="l59" cert="high">Bourges</placeName>.</note>

Note: The footnote refers to the travel plans of Geßner and Fries, who were heading to Bourges.
<note xml:id="fn49" type="footnote" n="49"><persName ref="p8437" cert="high">Erasmus</persName>, den Sohn des Verfassers.</note>, quos tuum deum et meum, ut potuisti, defendisti.</s>
				</p>
			</div>
		</body>
	</text>
</TEI>

Footnote 49: Erasmus, the son of the author.
<note xml:id="fn3" type="footnote" n="3"><persName ref="p1283" cert="high">Geßner</persName> and <persName ref="p1214" cert="high">Fries</persName> were on their way to <placeName ref="l59" cert="high">Bourges</placeName>.</note>

Note: The footnote refers to the fact that Geßner and Fries were on their way to Bourges, which is a city in France.
<note xml:id="fn3" type="footnote" n="3"><persName 

KeyboardInterrupt: 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!python COT_instruct.py meta-llama/Meta-Llama-3-8B-Instruct ../../data/dataset_en_split_dev.json instruct_prompt2.jsonl --few_shot examples.jsonl

new_
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100% 4/4 [01:20<00:00, 20.14s/it]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
download done
  0% 0/400 [00:00<?, ?it/s]2024-05-16 15:40:06.476520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [ ]:
import torch
torch.cuda.empty_cache()